<a href="https://colab.research.google.com/github/hasan-sh/advanced-nlp/blob/main/Firstmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [4]:
!git clone https://github.com/hasan-sh/advanced-nlp.git

fatal: destination path 'advanced-nlp' already exists and is not an empty directory.


In [5]:
!pwd

/content


In [6]:
train_file = '/content/advanced-nlp/data/en_ewt-up-train.conllu'
test_file = '/content/advanced-nlp/data/en_ewt-up-test.conllu'

In [7]:
def read_data(file_path, save_to_csv=False):
    """
    This function reads a CoNLL-U format file and converts it into a pandas DataFrame.
    Each row in the DataFrame corresponds to a token in the file, and columns
    correspond to different features of the token, such as the token itself, its lemma, 
    part-of-speech tag, and syntactic dependency information.
    
    Parameters:
    file_path (str): The path to the input CoNLL-U format file.
    save_to_csv (bool): A boolean flag indicating whether to save the resulting DataFrame 
                        to a CSV file. Default is False.
                        
    Returns:
    df (pandas.DataFrame): A pandas DataFrame containing the token-level information from
                           the input file.
    """
    
    # Open and read the input file
    with open(file_path, 'r', encoding='utf-8') as f:
        train_data = f.read()
    
    # Split the file into individual documents, each separated by a blank line
    data = []
    for doc_i, doc in enumerate(train_data.split('\n\n')):
        doc = doc.split('\n')
        sentences = ''
        for line in doc:
            # Skip lines starting with '#' (comment lines)
            if line and line[0] != '#':
                line = line.split('\t')
                line.insert(0, str(doc_i))
                sentences += '\t'.join(line) + '\n'
        data.append(sentences)
    
    # Create a pandas DataFrame from the token-level data
    train_df = pd.DataFrame([x.split('\t') for sent in data for x in sent.split('\n') if x])
    
    # Rename the columns of the DataFrame
    train_df = train_df.rename(columns={
        0:'sent_id', 
        1:'token_id', 
        2:'token', 
        3:'lemma', 
        4:'POS', 
        5:'uni_POS',
        6:'morph_type', 
        7:'distance_head', 
        8:'dep_label', 
        9:'dep_rel', 
        10:'space', 
        11:'probbank'
    })
    
    # Convert the DataFrame from wide to long format
    df = train_df.melt(
        id_vars=[i for i in train_df.columns[:12]], 
        var_name="notneeded", 
        value_name="target"
    )
    
    # Drop the 'notneeded' column and any rows that contain missing values
    df["sent_id"]=df['sent_id'].str.cat((df['notneeded'].astype(int)-12).astype(str) , sep="_" )
    df.drop(['notneeded'], axis=1, inplace=True)
    df = df[df['target'].notna()]
    
    # Optionally save the resulting DataFrame to a CSV file
    if save_to_csv:
        df.to_csv('/content/advanced-nlp/data/test.tsv', sep='\t', index=False)
    
    # Return the resulting DataFrame
    return df

In [8]:
x = read_data(test_file, save_to_csv=True)

In [9]:
z = read_data(train_file,save_to_csv=True)

##WITH ONLY TOKEN

In [38]:
df=z[["sent_id","token","target"]]
df_test=x[["sent_id","token","target"]]

In [39]:
df = df.assign(label=[0 if target=="_" or target=="V" else 1 for target in df['target']])
df_test = df_test.assign(label=[0 if target=="_" or target=="V" else 1 for target in df_test['target']])
#DROP TARGET HERE

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [41]:
vectorizer = CountVectorizer()
X_text = vectorizer.fit_transform(df['token'])

y = df['label']

# Create logistic regression model and fit it to the data
logreg = LogisticRegression()
logreg.fit(X_text, y)


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [42]:
X_test_text = vectorizer.transform(df_test["token"])
y_pred = logreg.predict(X_test_text)

-------*EVALUAATION*

In [43]:
from sklearn.metrics import accuracy_score

In [44]:
accuracy = accuracy_score(df_test["label"], y_pred)


print("Accuracy score:", accuracy)

Accuracy score: 0.888663967611336


In [45]:
y

0          0
1          0
2          0
3          0
4          0
          ..
7077906    0
7077907    0
7077908    1
7077909    0
7077910    1
Name: label, Length: 1035928, dtype: int64

##WITH FEATURES

In [53]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [61]:
df=z
df_test=x

In [62]:
df = df.assign(label=[0 if target=="_" or target=="V" else 1 for target in df['target']])
df_test = df_test.assign(label=[0 if target=="_" or target=="V" else 1 for target in df_test['target']])


In [63]:
cols=[ 'lemma', 'POS', 'uni_POS', 'morph_type',
       'distance_head', 'dep_label', 'dep_rel', 'space', 'probbank']

In [70]:
ct = ColumnTransformer([
    ('tfidf', CountVectorizer(), 'token'),
    ('onehot', OneHotEncoder(), cols)
])

# Define the logistic regression model
lr = LogisticRegression()

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', ct),
    ('classifier', lr)
])

# Split the data into X and y
X = df.drop('label', axis=1)
y = df['label']

# Fit the pipeline to the data
pipeline.fit(X, y)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('tfidf', CountVectorizer(),
                                                  'token'),
                                                 ('onehot', OneHotEncoder(),
                                                  ['lemma', 'POS', 'uni_POS',
                                                   'morph_type',
                                                   'distance_head', 'dep_label',
                                                   'dep_rel', 'space',
                                                   'probbank'])])),
                ('classifier', LogisticRegression())])

In [71]:
X_test=df_test.drop('label', axis=1)
y_test=df_test['label']
# Evaluate the model on the test data
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)


print("Accuracy score:", accuracy)


ValueError: ignored